In [ ]:
import numpy as np
from PIL import Image
import sys
from typing import List, Tuple

In [ ]:
# Configuration constants
DESIRED_SCALE_FACTOR = 0.5  # Final dimension will be 50% of original
INPUT_IMAGE_PATH = "original.jpg"
OUTPUT_IMAGE_PATH = "resized.jpg"
SEAM_VISUALIZATION_PATH = "seams_visualization.jpg"

In [ ]:
#########################
# ENERGY CALCULATION
#########################
def convert_to_grayscale(image_array: np.ndarray) -> np.ndarray:
    """
    Convert an RGB image to grayscale using:
         Gray = 0.299*R + 0.587*G + 0.114*B
    """
    return (0.299 * image_array[:, :, 0] +
            0.587 * image_array[:, :, 1] +
            0.114 * image_array[:, :, 2]).astype(np.float64)

def compute_energy(grayscale: np.ndarray) -> np.ndarray:
    """
    Compute the energy map of a grayscale image using:
         e₁ = |∂ₓI| + |∂ᵧI|
    For interior pixels, we approximate:
         ∂ₓI = I(i, j+1) - I(i, j-1)
         ∂ᵧI = I(i+1, j) - I(i-1, j)
    Border pixels are set to 1000.
    """
    h, w = grayscale.shape
    energy = np.zeros((h, w), dtype=np.float64)
    # Set borders
    energy[0, :] = 1000
    energy[h-1, :] = 1000
    energy[:, 0] = 1000
    energy[:, w-1] = 1000
    # Compute energy for interior pixels using slicing:
    dx = np.abs(grayscale[1:h-1, 2:w] - grayscale[1:h-1, 0:w-2])
    dy = np.abs(grayscale[2:h, 1:w-1] - grayscale[0:h-2, 1:w-1])
    energy[1:h-1, 1:w-1] = dx + dy
    return energy

In [ ]:
#########################
# VERTICAL SEAM CARVING
#########################
def find_vertical_seam(energy: np.ndarray) -> np.ndarray:
    """
    Find the vertical seam with the minimum cumulative energy.
    Returns a 1D array of length h; seam[i] is the column to remove at row i.
    """
    h, w = energy.shape
    M = np.copy(energy)
    backtrack = np.zeros((h, w), dtype=int)
    # Build cumulative energy map row by row
    for i in range(1, h):
        for j in range(w):
            if j == 0:
                idx = np.argmin(M[i-1, j:j+2])
                min_energy = M[i-1, j + idx]
                backtrack[i, j] = j + idx
            elif j == w - 1:
                idx = np.argmin(M[i-1, j-1:j+1])
                min_energy = M[i-1, j-1 + idx]
                backtrack[i, j] = j - 1 + idx
            else:
                idx = np.argmin(M[i-1, j-1:j+2])
                min_energy = M[i-1, j-1 + idx]
                backtrack[i, j] = j - 1 + idx
            M[i, j] += min_energy
    # Backtrack to find the seam
    seam = np.zeros(h, dtype=int)
    seam[h-1] = np.argmin(M[h-1])
    for i in range(h-2, -1, -1):
        seam[i] = backtrack[i+1, seam[i+1]]
    return seam

def remove_vertical_seam(image: np.ndarray, indices_map: np.ndarray, seam: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Remove the vertical seam from the image and update the indices map.
    - image: array of shape (h, w, 3)
    - indices_map: array of shape (h, w) tracking original column positions.
    - seam: 1D array of length h with the column index to remove per row.
    Returns new_image (h, w-1, 3) and updated indices_map.
    """
    h, w, _ = image.shape
    new_image = np.zeros((h, w-1, 3), dtype=image.dtype)
    new_indices = np.zeros((h, w-1), dtype=indices_map.dtype)
    for i in range(h):
        col = seam[i]
        new_image[i, :col] = image[i, :col]
        new_image[i, col:] = image[i, col+1:]
        new_indices[i, :col] = indices_map[i, :col]
        new_indices[i, col:] = indices_map[i, col+1:]
    return new_image, new_indices

def carve_vertical(image: Image.Image, desired_width: int) -> Tuple[np.ndarray, List[List[Tuple[int, int]]]]:
    """
    Iteratively remove vertical seams until the image's width equals desired_width.
    Returns:
      - The carved image as an array.
      - A list of removed seam coordinates (each seam is a list of (row, original_column) tuples).
    """
    image_np = np.array(image)
    h, w, _ = image_np.shape
    if w <= desired_width:
        raise ValueError("Image width must be greater than desired width.")
    indices_map = np.tile(np.arange(w), (h, 1))
    seams_removed: List[List[Tuple[int, int]]] = []
    current_image = image_np
    current_w = w
    while current_w > desired_width:
        gray = convert_to_grayscale(current_image)
        energy = compute_energy(gray)
        seam = find_vertical_seam(energy)
        # Record seam coordinates in original image space using indices_map.
        seam_coords = [(i, int(indices_map[i, seam[i]])) for i in range(h)]
        seams_removed.append(seam_coords)
        current_image, indices_map = remove_vertical_seam(current_image, indices_map, seam)
        current_w -= 1
    return current_image, seams_removed

In [ ]:
#########################
# HORIZONTAL SEAM CARVING
#########################
def carve_horizontal(image: Image.Image, desired_height: int) -> Tuple[np.ndarray, List[List[Tuple[int, int]]]]:
    """
    Perform horizontal seam carving by transposing the image, carving vertically,
    then transposing back.
    Returns the carved image and list of removed seam coordinates (in original orientation).
    """
    np_img = np.array(image)
    # Transpose so that rows become columns
    transposed = np.transpose(np_img, (1, 0, 2))
    transposed_image = Image.fromarray(transposed)
    desired_width = desired_height  # since transposed width equals original height
    carved_transposed, seams_removed_transposed = carve_vertical(transposed_image, desired_width)
    # Transpose back to get the carved horizontal image
    carved_image = np.transpose(carved_transposed, (1, 0, 2))
    # Convert seam coordinates back to original orientation:
    # In the transposed image, a coordinate (row, col) corresponds to (col, original_height - row - 1)
    transformed_seams = []
    for seam in seams_removed_transposed:
        transformed = [(col, transposed.shape[0] - row - 1) for row, col in seam]
        transformed_seams.append(transformed)
    return carved_image, transformed_seams

In [ ]:
#########################
# AUTOMATIC DUAL-DIRECTION CARVING
#########################
def carve_image(image: Image.Image, scale_factor: float) -> Tuple[Image.Image, Image.Image]:
    """
    Automatically reduce both width and height based on scale_factor.
    For example, a scale_factor of 0.5 reduces both dimensions to 50% of the original.

    Returns a tuple:
      (resized image, seam visualization image)
    The seam visualization image is the original image with all removed seam pixels marked in red.
    """
    image_np = np.array(image)
    h, w, _ = image_np.shape
    desired_width = int(w * scale_factor)
    desired_height = int(h * scale_factor)

    # Calculate number of seams to remove in each dimension
    cols_to_remove = w - desired_width
    rows_to_remove = h - desired_height

    # Keep a copy of the original image for seam visualization.
    original_image = image_np.copy()

    vertical_seams: List[List[Tuple[int, int]]] = []
    horizontal_seams: List[List[Tuple[int, int]]] = []

    # Remove horizontal seams first (rows)
    carved_image = image_np
    if rows_to_remove > 0:
        carved_image, horizontal_seams = carve_horizontal(Image.fromarray(carved_image), desired_height)
    # Remove vertical seams next (columns)
    if cols_to_remove > 0:
        carved_image, vertical_seams = carve_vertical(Image.fromarray(carved_image), desired_width)

    # Mark seams on the original image.
    marked_image = original_image.copy()
    # Mark vertical seams
    for seam in vertical_seams:
        for (row, col) in seam:
            if 0 <= row < marked_image.shape[0] and 0 <= col < marked_image.shape[1]:
                marked_image[row, col] = [255, 0, 0]  # red
    # Mark horizontal seams
    for seam in horizontal_seams:
        for (row, col) in seam:
            if 0 <= row < marked_image.shape[0] and 0 <= col < marked_image.shape[1]:
                marked_image[row, col] = [255, 0, 0]  # red

    resized_img = Image.fromarray(carved_image)
    vis_img = Image.fromarray(marked_image)
    return resized_img, vis_img


In [ ]:
#########################
# MAIN FUNCTION
#########################
def main():
    # Command-line usage:
    # python seam_carving.py input.jpg resized.jpg seams_vis.jpg
    if len(sys.argv) >= 4:
        input_path = sys.argv[1]
        output_path = sys.argv[2]
        seams_vis_path = sys.argv[3]
    else:
        input_path = INPUT_IMAGE_PATH
        output_path = OUTPUT_IMAGE_PATH
        seams_vis_path = SEAM_VISUALIZATION_PATH

    # Load the image.
    input_image = Image.open(input_path).convert("RGB")

    # Warn if dimensions exceed 800x800.
    if input_image.width > 800 or input_image.height > 800:
        print("Warning: Image dimensions exceed 800x800. Make sure your algorithm handles such sizes.")

    # Check that dimensions allow reduction by half.
    if input_image.width < (input_image.width // 2) or input_image.height < (input_image.height // 2):
        raise ValueError("Image dimensions must be sufficiently large for reduction by half.")

    # Optionally adjust recursion limit.
    sys.setrecursionlimit(input_image.height + 10)

    # Save grayscale and energy visualization for reference.
    input_np = np.array(input_image)
    gray_np = convert_to_grayscale(input_np)
    gray_img = Image.fromarray(gray_np.astype(np.uint8))
    gray_img.save("grayscale.jpg", quality=100)

    energy = compute_energy(gray_np)
    max_energy = np.max(energy)
    energy_vis = np.zeros((energy.shape[0], energy.shape[1], 3), dtype=np.uint8)
    for i in range(energy.shape[0]):
        for j in range(energy.shape[1]):
            val = int((energy[i, j] * 255) / max_energy) if max_energy > 0 else 0
            energy_vis[i, j] = (val, val, val)
    energy_img = Image.fromarray(energy_vis)
    energy_img.save("energy.jpg", quality=100)

    # Perform automatic seam carving in both directions.
    resized_img, seams_vis_img = carve_image(input_image, DESIRED_SCALE_FACTOR)
    resized_img.save(output_path, quality=100)
    seams_vis_img.save(seams_vis_path, quality=100)

    print("Seam carving completed.")
    print("Resized image saved as:", output_path)
    print("Seam visualization saved as:", seams_vis_path)

if __name__ == "__main__":
    main()